In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
vac  = spark.read.parquet("/dados10t/datalake/standard/dados_datasus/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/")

In [42]:
vac.take(3)

[Row(d_sexo='M', d_codibge='290210', d_idade=33.0, d_racacor='PARDA', d_paciente_endereco_uf='BA', d_total_regitros_dedup=2.0, d1_nome_vacina='Covid-19-AstraZeneca', d1_data_aplicacao=datetime.date(2021, 5, 28), d1_estab_mun_codigo='290210', d1_estab_valor='6848079', d1_vacina_grupoAtendimento_nome='Doenças Cardiovasculares e Cerebrovasculares', d1_vacina_categoria_nome='Comorbidades', d1_vacina_dose='1ª Dose', d2_nome_vacina='Covid-19-AstraZeneca', d2_data_aplicacao=datetime.date(2021, 8, 26), d2_estab_mun_codigo='290210', d2_estab_valor='6848079', d2_vacina_grupoAtendimento_nome='Doenças Cardiovasculares e Cerebrovasculares', d2_vacina_categoria_nome='Comorbidades', d2_vacina_dose='2ª Dose', d3_nome_vacina=None, d3_data_aplicacao=None, d3_estab_mun_codigo=None, d3_estab_valor=None, d3_vacina_grupoAtendimento_nome=None, d3_vacina_categoria_nome=None, d3_vacina_dose=None, d4_nome_vacina=None, d4_data_aplicacao=None, d4_estab_mun_codigo=None, d4_estab_valor=None, d4_vacina_grupoAtendime

In [41]:
vac.printSchema()

root
 |-- d_sexo: string (nullable = true)
 |-- d_codibge: string (nullable = true)
 |-- d_idade: double (nullable = true)
 |-- d_racacor: string (nullable = true)
 |-- d_paciente_endereco_uf: string (nullable = true)
 |-- d_total_regitros_dedup: double (nullable = true)
 |-- d1_nome_vacina: string (nullable = true)
 |-- d1_data_aplicacao: date (nullable = true)
 |-- d1_estab_mun_codigo: string (nullable = true)
 |-- d1_estab_valor: string (nullable = true)
 |-- d1_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d1_vacina_categoria_nome: string (nullable = true)
 |-- d1_vacina_dose: string (nullable = true)
 |-- d2_nome_vacina: string (nullable = true)
 |-- d2_data_aplicacao: date (nullable = true)
 |-- d2_estab_mun_codigo: string (nullable = true)
 |-- d2_estab_valor: string (nullable = true)
 |-- d2_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d2_vacina_categoria_nome: string (nullable = true)
 |-- d2_vacina_dose: string (nullable = true)
 |-- d3_nome_vacin

In [3]:
srag  = spark.read.parquet("/dados10t/datalake/standard/dados_datasus/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")

In [4]:
sg  = spark.read.parquet("/dados10t/datalake/standard/dados_datasus/SG/sg_long_09072022_v9.parquet/")

In [5]:
vac.groupBy("d_sexo").count().orderBy(F.desc("count")).show(300,False) 

+------+--------+
|d_sexo|count   |
+------+--------+
|F     |95510810|
|M     |88245694|
|I     |3437    |
+------+--------+



In [6]:
srag.groupBy("cs_sexo").count().orderBy(F.desc("count")).show(300,False) 

+-------+-------+
|cs_sexo|count  |
+-------+-------+
|M      |1344547|
|F      |1142513|
|I      |25     |
+-------+-------+



In [7]:
sg.groupBy("sexo_2").count().orderBy(F.desc("count")).show(300,False) 

+----------+--------+
|sexo_2    |count   |
+----------+--------+
|Feminino  |45511625|
|Masculino |37301161|
|Indefinido|82953   |
|null      |780     |
+----------+--------+



In [5]:
def sex_harmo(var,idioma="pt-br"): #
    if idioma == "pt-br":
        if var =="M" or var == "Masculino":
            return "Masculino"
        elif var == 'F' or var == "Feminino":
            return "Feminino"
    if idioma == "en":
        if var =="M" or var == "Masculino":
            return "Male"
        elif var == "F" or var == "Feminino":
            return "Female"
    
        
        
def raca_harmo(var,idioma="pt-br"): #
    if idioma == "pt-br":
        pass
    elif idioma == "en":
        if var =="BRANCA":
            return "White"
        elif var == "PRETA" or var == "PARDA":
            return "Black"
        elif var == "INDIGENA":
            return "American Indian"
        elif var == "AMARELA":
            return "Asian"
    


In [6]:
udf_sex_omop = F.udf(sex_harmo,StringType() )

udf_race_omop = F.udf(raca_harmo,StringType() )

In [31]:
vac = vac.withColumn("lang",F.lit("en"))
sg = sg.withColumn("lang",F.lit("en"))
srag = srag.withColumn("lang",F.lit("en"))

In [8]:
vac = vac.withColumn("sexo_omop",udf_sex_omop(F.col("d_sexo")))
vac = vac.withColumn("sexo_omop_en", udf_sex_omop( F.col("d_sexo"),F.col("lang") ))


sg = sg.withColumn("sexo_omop",udf_sex_omop(F.col("sexo_2")))
sg = sg.withColumn("sexo_omop_en", udf_sex_omop( "sexo_2","lang" ))


srag = srag.withColumn("sexo_omop",udf_sex_omop(F.col("cs_sexo")))
srag = srag.withColumn("sexo_omop_en", udf_sex_omop( "cs_sexo","lang" ))

In [12]:
vac.groupBy("sexo_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+--------+
|sexo_omop|count   |
+---------+--------+
|Feminino |95510810|
|Masculino|88245694|
|null     |3437    |
+---------+--------+



In [13]:
vac.groupBy("sexo_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+------------+--------+
|sexo_omop_en|count   |
+------------+--------+
|Female      |95510810|
|Male        |88245694|
|null        |3437    |
+------------+--------+



In [14]:
sg.groupBy("sexo_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+--------+
|sexo_omop|count   |
+---------+--------+
|Feminino |45511625|
|Masculino|37301161|
|null     |83733   |
+---------+--------+



In [15]:
sg.groupBy("sexo_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+------------+--------+
|sexo_omop_en|count   |
+------------+--------+
|Female      |45511625|
|Male        |37301161|
|null        |83733   |
+------------+--------+



In [16]:
srag.groupBy("sexo_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+-------+
|sexo_omop|count  |
+---------+-------+
|Masculino|1344547|
|Feminino |1142513|
|null     |25     |
+---------+-------+



In [17]:
srag.groupBy("sexo_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+------------+-------+
|sexo_omop_en|count  |
+------------+-------+
|Male        |1344547|
|Female      |1142513|
|null        |25     |
+------------+-------+



In [9]:
def raca_harmo(var,idioma="pt-br"): #
    if var:
        var = var.upper()
        if idioma == "pt-br":
            return var
        elif idioma == "en":
            if var =="BRANCA":
                return "White"
            elif var == "PRETA" or var == "PARDA":
                return "Black"
            elif var == "INDIGENA":
                return "American Indian"
            elif var == "AMARELA":
                return "Asian"
            
            
def fix_raca_srag(raca):
    raca = str(raca)
    if raca:
        if raca == "1":
            return "BRANCA"
        elif raca == "2":
            return "PRETA"
        elif raca == "3":
            return "AMARELA"
        elif raca == "4":
            return "PARDA"
        elif raca == "5":
            return "INDIGENA"
    


In [10]:
udf_race_omop = F.udf(raca_harmo,StringType() )
udf_fix_raca_srag = F.udf(fix_raca_srag,StringType() )

In [11]:
vac = vac.withColumn("raca_omop",udf_race_omop(F.col("d_racacor")))
vac = vac.withColumn("raca_omop_en", udf_race_omop( F.trim(F.col("d_racacor")),F.col("lang") ))


sg = sg.withColumn("raca_omop",udf_race_omop(F.col("racacor_2")))
sg = sg.withColumn("raca_omop_en", udf_race_omop( "racacor_2","lang" ))


srag = srag.withColumn("raca_omop",udf_fix_raca_srag(F.col("cs_raca")))
srag = srag.withColumn("raca_omop_en", udf_race_omop( "raca_omop","lang" ))

In [21]:
vac.groupBy("raca_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+--------+
|raca_omop|count   |
+---------+--------+
|BRANCA   |61517395|
|null     |45074568|
|PARDA    |42199385|
|AMARELA  |26604921|
|PRETA    |7806738 |
|INDIGENA |556934  |
+---------+--------+



In [22]:
vac.groupBy("raca_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+---------------+--------+
|raca_omop_en   |count   |
+---------------+--------+
|White          |61517395|
|Black          |50006123|
|null           |45074568|
|Asian          |26604921|
|American Indian|556934  |
+---------------+--------+



In [23]:
sg.groupBy("raca_omop").count().orderBy(F.desc("count")).show(300,False) 

+-------------+--------+
|raca_omop    |count   |
+-------------+--------+
|BRANCA       |31615421|
|PARDA        |25570173|
|null         |19866924|
|PRETA        |3218746 |
|AMARELA      |2456329 |
|INDIGENA     |168924  |
|NÃO INFORMADO|2       |
+-------------+--------+



In [24]:
sg.groupBy("raca_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+---------------+--------+
|raca_omop_en   |count   |
+---------------+--------+
|White          |31615421|
|Black          |28788919|
|null           |19866926|
|Asian          |2456329 |
|American Indian|168924  |
+---------------+--------+



In [25]:
srag.groupBy("cs_raca").count().orderBy(F.desc("count")).show(300,False) 

+-------+-------+
|cs_raca|count  |
+-------+-------+
|1      |1098563|
|4      |821187 |
|9      |403505 |
|2      |107781 |
|null   |27250  |
|3      |24113  |
|5      |4686   |
+-------+-------+



In [26]:
srag.groupBy("raca_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+-------+
|raca_omop|count  |
+---------+-------+
|BRANCA   |1098563|
|PARDA    |821187 |
|null     |430755 |
|PRETA    |107781 |
|AMARELA  |24113  |
|INDIGENA |4686   |
+---------+-------+



In [27]:
srag.groupBy("raca_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+---------------+-------+
|raca_omop_en   |count  |
+---------------+-------+
|White          |1098563|
|Black          |928968 |
|null           |430755 |
|Asian          |24113  |
|American Indian|4686   |
+---------------+-------+



In [12]:
#teste2 = srag.groupBy("id_vigvac").agg(F.collect_set("sexo_omop_en").alias("sexo_set"),F.collect_list("sexo_omop_en").alias("sexo_list") ).cache()

In [13]:
#teste2 = teste2.withColumn("qtd_sexo",F.size("sexo_list"))
#teste2 = teste2.withColumn("qtd_sexo_diff",F.size("sexo_set"))

In [14]:
#teste2.orderBy(F.desc("qtd_sexo_diff")).show(30,False) #validar escolha do mais frequente

In [15]:
vac.printSchema()

root
 |-- d_sexo: string (nullable = true)
 |-- d_codibge: string (nullable = true)
 |-- d_idade: double (nullable = true)
 |-- d_racacor: string (nullable = true)
 |-- d_paciente_endereco_uf: string (nullable = true)
 |-- d_total_regitros_dedup: double (nullable = true)
 |-- d1_nome_vacina: string (nullable = true)
 |-- d1_data_aplicacao: date (nullable = true)
 |-- d1_estab_mun_codigo: string (nullable = true)
 |-- d1_estab_valor: string (nullable = true)
 |-- d1_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d1_vacina_categoria_nome: string (nullable = true)
 |-- d1_vacina_dose: string (nullable = true)
 |-- d2_nome_vacina: string (nullable = true)
 |-- d2_data_aplicacao: date (nullable = true)
 |-- d2_estab_mun_codigo: string (nullable = true)
 |-- d2_estab_valor: string (nullable = true)
 |-- d2_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d2_vacina_categoria_nome: string (nullable = true)
 |-- d2_vacina_dose: string (nullable = true)
 |-- d3_nome_vacin

In [16]:
srag_counts_sexo =  srag.groupBy("id_vigvac","sexo_omop").count().cache()
srag_counts_raca =  srag.groupBy("id_vigvac","raca_omop").count().cache()


srag_grouped_sexo = srag_counts_sexo.orderBy(['id_vigvac', F.desc('count')])
srag_grouped_raca = srag_counts_raca.orderBy(['id_vigvac', F.desc('count')])

srag_result_sexo  = srag_grouped_sexo.groupBy('id_vigvac').agg(F.first("sexo_omop").alias("sexo"),F.first("count").alias("count")).cache()
srag_result_raca  = srag_grouped_raca.groupBy('id_vigvac').agg(F.first("raca_omop").alias("raca"),F.first("count").alias("count")).cache()

In [17]:
sg_counts_sexo =  sg.groupBy("id_vigvac","sexo_omop").count().cache()
sg_counts_raca =  sg.groupBy("id_vigvac","raca_omop").count().cache()


sg_grouped_sexo = sg_counts_sexo.orderBy(['id_vigvac', F.desc('count')])
sg_grouped_raca = sg_counts_raca.orderBy(['id_vigvac', F.desc('count')])

sg_result_sexo  = sg_grouped_sexo.groupBy('id_vigvac').agg(F.first("sexo_omop").alias("sexo"),F.first("count").alias("count")).cache()
sg_result_raca  = sg_grouped_raca.groupBy('id_vigvac').agg(F.first("raca_omop").alias("raca"),F.first("count").alias("count")).cache()

In [18]:
vac_counts_sexo =  vac.groupBy("id_vigvac","sexo_omop").count().cache()
vac_counts_raca =  vac.groupBy("id_vigvac","raca_omop").count().cache()


vac_grouped_sexo = vac_counts_sexo.orderBy(['id_vigvac', F.desc('count')])
vac_grouped_raca = vac_counts_raca.orderBy(['id_vigvac', F.desc('count')])

vac_result_sexo  = vac_grouped_sexo.groupBy('id_vigvac').agg(F.first("sexo_omop").alias("sexo"),F.first("count").alias("count")).cache()
vac_result_raca  = vac_grouped_raca.groupBy('id_vigvac').agg(F.first("raca_omop").alias("raca"),F.first("count").alias("count")).cache()

In [19]:
print("srag sexo {} e srag raça {}".format(srag_result_sexo.count(),srag_result_raca.count() ))

srag sexo 2320559 e srag raça 2320559


In [38]:
print("SG sexo {} e SG raça {}".format(sg_result_sexo.count(),sg_result_raca.count() ))

SG sexo 53109190 e SG raça 53109190


In [39]:
print("vac sexo {} e vac raça {}".format(vac_result_sexo.count(),vac_result_raca.count()))

vac sexo 183759941 e vac raça 183759941


In [20]:
srag_termos = srag_result_sexo.join(srag_result_raca,['id_vigvac'],'inner')
sg_termos = sg_result_sexo.join(sg_result_raca,['id_vigvac'],'inner')
vac_termos = vac_result_sexo.join(vac_result_raca,['id_vigvac'],'inner')

In [42]:
print("SRAG {}".format(srag_termos.count()))
print("VAC {}".format(sg_termos.count()))
print("SG {}".format(vac_termos.count()))

SRAG 2320559


VAC 53109190


SG 183759941


In [21]:
for variavel in srag_termos.columns:
    if variavel == "id_vigvac":
        pass
    else:
        srag_termos = srag_termos.withColumnRenamed(variavel,"srag_"+variavel)

In [22]:
for variavel in sg_termos.columns:
    if variavel == "id_vigvac":
        pass
    else:
        sg_termos = sg_termos.withColumnRenamed(variavel,"sg_"+variavel)

In [23]:
for variavel in vac_termos.columns:
    if variavel == "id_vigvac":
        pass
    else:
        vac_termos = vac_termos.withColumnRenamed(variavel,"vac_"+variavel)

In [24]:
srag_termos_v2 = srag_termos.select("id_vigvac","srag_sexo","srag_raca")
sg_termos_v2 = sg_termos.select("id_vigvac","sg_sexo","sg_raca")
vac_termos_v2 = vac_termos.select("id_vigvac","vac_sexo","vac_raca")

In [25]:
pessoas_termos = vac_termos_v2.join(sg_termos_v2,['id_vigvac'],'full')
pessoas_termos = pessoas_termos.join(srag_termos_v2,['id_vigvac'],'full')

In [55]:
pessoas_termos.count()

188537327

In [56]:
pessoas_termos.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vac_sexo: string (nullable = true)
 |-- vac_raca: string (nullable = true)
 |-- sg_sexo: string (nullable = true)
 |-- sg_raca: string (nullable = true)
 |-- srag_sexo: string (nullable = true)
 |-- srag_raca: string (nullable = true)



In [26]:
pessoas_termos = pessoas_termos.withColumn("sexo_omop",F.coalesce("srag_sexo","vac_sexo","sg_sexo") )
pessoas_termos = pessoas_termos.withColumn("raca_omop",F.coalesce("srag_raca","vac_raca","sg_raca") )

In [27]:
pessoas_termos.groupBy("sexo_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+--------+
|sexo_omop|count   |
+---------+--------+
|Feminino |97691185|
|Masculino|90840855|
|null     |5287    |
+---------+--------+



In [28]:
pessoas_termos.groupBy("raca_omop").count().orderBy(F.desc("count")).show(300,False) 

+---------+--------+
|raca_omop|count   |
+---------+--------+
|BRANCA   |67172103|
|PARDA    |47435553|
|null     |38292655|
|AMARELA  |26676084|
|PRETA    |8369733 |
|INDIGENA |591199  |
+---------+--------+



In [30]:
pessoas_termos.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vac_sexo: string (nullable = true)
 |-- vac_raca: string (nullable = true)
 |-- sg_sexo: string (nullable = true)
 |-- sg_raca: string (nullable = true)
 |-- srag_sexo: string (nullable = true)
 |-- srag_raca: string (nullable = true)
 |-- sexo_omop: string (nullable = true)
 |-- raca_omop: string (nullable = true)



In [32]:
pessoas_termos = pessoas_termos.withColumn("lang",F.lit("en"))

In [33]:
pessoas_termos = pessoas_termos.withColumn("raca_omop_en", udf_race_omop( F.trim(F.col("raca_omop")),F.col("lang") ))
pessoas_termos = pessoas_termos.withColumn("sexo_omop_en", udf_sex_omop( F.col("sexo_omop"),F.col("lang") ))

In [34]:
pessoas_termos.groupBy("sexo_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+------------+--------+
|sexo_omop_en|count   |
+------------+--------+
|Female      |97691185|
|Male        |90840855|
|null        |5287    |
+------------+--------+



In [35]:
pessoas_termos.groupBy("raca_omop_en").count().orderBy(F.desc("count")).show(300,False) 

+---------------+--------+
|raca_omop_en   |count   |
+---------------+--------+
|White          |67172103|
|Black          |55805286|
|null           |38292655|
|Asian          |26676084|
|American Indian|591199  |
+---------------+--------+



In [36]:
sexo_lista = pessoas_termos.groupBy("sexo_omop").count().orderBy(F.desc("count")).cache()
raca_lista = pessoas_termos.groupBy("raca_omop").count().orderBy(F.desc("count")).cache()

In [40]:
raca_lista.show()

+---------+--------+
|raca_omop|   count|
+---------+--------+
|   BRANCA|67172103|
|    PARDA|47435553|
|     null|38292655|
|  AMARELA|26676084|
|    PRETA| 8369733|
| INDIGENA|  591199|
+---------+--------+



In [39]:
sexo_lista.show()

+---------+--------+
|sexo_omop|   count|
+---------+--------+
| Feminino|97691185|
|Masculino|90840855|
|     null|    5287|
+---------+--------+



In [69]:
sexo_lista_termos = sexo_lista.select("sexo_omop").withColumnRenamed("sexo_omop","termo")
raca_lista_termos = raca_lista.select("raca_omop").withColumnRenamed("raca_omop","termo")

In [71]:
sexo_lista_termos.show()

+------+
| termo|
+------+
|Female|
|  Male|
|  null|
+------+



In [72]:
final = sexo_lista_termos.unionByName(raca_lista_termos)

In [73]:
final.toPandas().to_csv("/home/juracy.santos/lista_termos_tb_pessoa_omop.csv",sep=";")

In [74]:
pessoas_termos.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vac_sexo: string (nullable = true)
 |-- vac_raca: string (nullable = true)
 |-- sg_sexo: string (nullable = true)
 |-- sg_raca: string (nullable = true)
 |-- srag_sexo: string (nullable = true)
 |-- srag_raca: string (nullable = true)
 |-- sexo_omop: string (nullable = true)
 |-- raca_omop: string (nullable = true)



In [75]:
tb_pessoa = pessoas_termos.select("id_vigvac","sexo_omop","raca_omop")

In [82]:
mapeamento_sexo_raca = spark.read.csv("/dados10t/datalake/standard/omop/resultado_mapeamento_sexo_raca.csv",sep=",",header=True)
mapeamento_sexo_raca = mapeamento_sexo_raca.drop("invalid_reason","valid_start_date","valid_end_date","target_vocabulary_id")

In [83]:
mapeamento_sexo_raca.show(5)

+-----------+-----------------+--------------------+-----------------------+-----------------+
|source_code|source_concept_id|source_vocabulary_id|source_code_description|target_concept_id|
+-----------+-----------------+--------------------+-----------------------+-----------------+
|          3|                0|                   1|                  White|             8527|
|          6|                0|                   1|                  Asian|             8515|
|          4|                0|                   1|                  Black|         38003598|
|          0|                0|                   1|                 Female|             8532|
|          1|                0|                   1|                   Male|             8507|
+-----------+-----------------+--------------------+-----------------------+-----------------+
only showing top 5 rows



In [80]:
tb_pessoa.show(10)

+---------+---------+---------+
|id_vigvac|sexo_omop|raca_omop|
+---------+---------+---------+
|       26|   Female|    Black|
|       29|   Female|    White|
|      474|   Female|     null|
|      964|   Female|    White|
|     1677|   Female|    Black|
|     1697|   Female|    Asian|
|     1806|     Male|     null|
|     1950|     Male|    White|
|     2040|   Female|     null|
|     2214|   Female|    Asian|
+---------+---------+---------+
only showing top 10 rows



In [ ]:
#add conceitos e dados do mapeamento
tb_pessoa_v2 = tb_pessoa.join(mapeamento_sexo_raca,)

In [77]:
mapeamento_sexo_raca.printSchema()

root
 |-- source_code: string (nullable = true)
 |-- source_concept_id: string (nullable = true)
 |-- source_vocabulary_id: string (nullable = true)
 |-- source_code_description: string (nullable = true)
 |-- target_concept_id: string (nullable = true)
 |-- target_vocabulary_id: string (nullable = true)
 |-- valid_start_date: string (nullable = true)
 |-- valid_end_date: string (nullable = true)
 |-- invalid_reason: string (nullable = true)

